In [ ]:
# %%

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

import numpy as np
import pandas as pd

from sklearnex import patch_sklearn
patch_sklearn()

label = 'signature'
time_limit = 10*60       # max training time (seconds)
infer_limit = 1/10000    # prediction seconds per row
finalModel = 'WeightedEnsemble_L2'

data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'

train_data = TabularDataset(f'{data_url}train.csv')
test_data = TabularDataset(f'{data_url}test.csv')

pruning_test_data = test_data.sample(frac=0.5, random_state=2024)
final_test_data = test_data.drop(pruning_test_data.index)

train_data.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# %% 

# Detect features to prune
np.random.seed(2024)

pruning_predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)
pruning_predictor.fit(train_data, presets='medium_quality', time_limit = time_limit)  

pruning_leaders = pruning_predictor.leaderboard(pruning_test_data)
pruning_leaders

No path specified. Models will be saved in: "AutogluonModels\ag-20240826_084051"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       20.19 GB / 31.75 GB (63.6%)
Disk Space Avail:   758.57 GB / 952.44 GB (79.6%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels\ag-20240826_084051"
Train Data Rows:    10000
Train Data Columns: 18
Label Column:       signature
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 13
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    20654.08 MB
	Train Data (Original)  Memory Usage: 1.37 MB (0.0% of available memory)
	Inferring data type of each feature based

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.152299,-0.129025,log_loss,0.287824,0.054340,107.559362,0.004983,0.000000,0.305769,2,True,14
1,CatBoost,-0.159601,-0.150310,log_loss,0.036415,0.009543,77.029672,0.036415,0.009543,77.029672,1,True,8
2,NeuralNetFastAI,-0.175997,-0.166266,log_loss,0.046353,0.020436,12.628047,0.046353,0.020436,12.628047,1,True,3
3,XGBoost,-0.182194,-0.144754,log_loss,0.164519,0.012466,4.124128,0.164519,0.012466,4.124128,1,True,11
4,LightGBM,-0.190363,-0.161321,log_loss,0.048541,0.008974,2.549535,0.048541,0.008974,2.549535,1,True,5
5,RandomForestEntr,-0.199600,-0.194874,log_loss,0.138113,0.045467,1.075156,0.138113,0.045467,1.075156,1,True,7
6,NeuralNetTorch,-0.199837,-0.163288,log_loss,0.023427,0.012464,87.951883,0.023427,0.012464,87.951883,1,True,12
7,LightGBMXT,-0.202060,-0.178918,log_loss,0.131375,0.028423,4.157646,0.131375,0.028423,4.157646,1,True,4
8,RandomForestGini,-0.211661,-0.206937,log_loss,0.170108,0.045943,0.994664,0.170108,0.045943,0.994664,1,True,6
9,ExtraTreesEntr,-0.263108,-0.267746,log_loss,0.203410,0.048217,0.739895,0.203410,0.048217,0.739895,1,True,10


In [ ]:
# %%

df_pruning_features = pruning_predictor.feature_importance(pruning_test_data)
low_importance_features = df_pruning_features.query("importance < 0.001").index.to_list()
low_importance_features

These features in provided data are not utilized by the predictor and will be ignored: ['Symmetry_D8']
Computing feature importance via permutation shuffling for 17 features using 2500 rows with 5 shuffle sets...
	28.98s	= Expected runtime (5.8s per shuffle set)
c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-packages\sklearn\metrics\_classification.py:2969: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-packages\sklearn\metrics\_classification.py:2969: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-packages\sklearn\metrics\_classification.py:2969: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-p

['Symmetry_D6', 'Unnamed: 0', 'Symmetry_D3']

In [ ]:
# %%

final_train_data = train_data.drop(columns=low_importance_features)

In [ ]:
# %%

custom_hyperparameters = get_hyperparameter_config('zeroshot') # ['default', 'zeroshot', 'light', 'very_light', 'toy', 'multimodal']

custom_hyperparameters['LR'] = [
    {'multi_class':'multinomial', 'penalty':None, 'tol':1e-6, 'max_iter':10000,
        'ag_args': {'name_suffix': 'Base'}},
    {'multi_class':'multinomial', 'penalty':'l2', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Ridge'}},
    {'multi_class':'multinomial', 'penalty':'l1', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Lasso'}},
    {'multi_class':'multinomial', 'penalty':'elasticnet', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'ElasticNet'}}
    ]

custom_preset = {'auto_stack': True,
                'dynamic_stacking': False,
                'hyperparameters':custom_hyperparameters,
                'refit_full': False,
                'set_best_to_refit_full': False,
                'save_bag_folds': True,
                'time_limit': time_limit,
                'infer_limit': infer_limit}

In [ ]:
# %%

np.random.seed(2024)

predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)

predictor.fit(final_train_data, presets=custom_preset)  

No path specified. Models will be saved in: "AutogluonModels\ag-20240826_084424"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       20.07 GB / 31.75 GB (63.2%)
Disk Space Avail:   757.97 GB / 952.44 GB (79.6%)
Presets specified: [{'auto_stack': True, 'dynamic_stacking': False, 'hyperparameters': {'NN_TORCH': [{}, {'activation': 'elu', 'dropout_prob': 0.10077639529843717, 'hidden_size': 108, 'learning_rate': 0.002735937344002146, 'num_layers': 4, 'use_batchnorm': True, 'weight_decay': 1.356433327634438e-12, 'ag_args': {'name_suffix': '_r79', 'priority': -2}}, {'activation': 'elu', 'dropout_prob': 0.11897478034205347, 'hidden_size': 213, 'learning_rate': 0.0010474382260641949, 'num_layers': 4, 'use_batchnorm': False, 'weight_decay': 5.594471067786272e-10, 'ag_args': {'name_suf

In [ ]:
# %%

predictor.persist(models='all', max_memory=0.5) # improves prediction time, consumes more memory

df_leaders = predictor.leaderboard(final_test_data)

df_leaders.head(40)

Persisting 26 models in memory. Models will require 6.07% of memory.


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.129585,-0.130726,log_loss,0.772544,1.544052,418.385474,0.001993,0.002989,0.321915,3,True,26
1,LightGBMXT_BAG_L2,-0.130915,-0.174001,log_loss,0.737531,1.280824,314.858429,0.188370,0.567184,13.277578,2,True,18
2,ExtraTreesEntr_BAG_L2,-0.131844,-0.141450,log_loss,0.613029,0.956844,302.356629,0.063869,0.243203,0.775778,2,True,24
3,CatBoost_BAG_L2,-0.134827,-0.143819,log_loss,0.580056,0.772443,414.628627,0.030895,0.058803,113.047777,2,True,22
4,WeightedEnsemble_L2,-0.135982,-0.139773,log_loss,0.550158,0.716632,302.142980,0.000997,0.002992,0.562129,2,True,16
5,NeuralNetFastAI_BAG_L2,-0.137136,-0.150249,log_loss,0.898990,0.916963,322.505842,0.349829,0.203323,20.924991,2,True,17
6,RandomForestEntr_BAG_L2,-0.137723,-0.140167,log_loss,0.612948,0.992778,303.255224,0.063787,0.279137,1.674374,2,True,21
7,RandomForestGini_BAG_L2,-0.139145,-0.150694,log_loss,0.651892,1.000634,303.249269,0.102731,0.286993,1.668419,2,True,20
8,ExtraTreesGini_BAG_L2,-0.141437,-0.143658,log_loss,0.612000,0.959919,302.565630,0.062839,0.246279,0.984780,2,True,23
9,XGBoost_BAG_L2,-0.142561,-0.151890,log_loss,0.643844,0.887140,320.539421,0.094683,0.173499,18.958570,2,True,25


In [ ]:
# %%

df_leaders.tail(20)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
6,RandomForestEntr_BAG_L2,-0.137723,-0.140167,log_loss,0.612948,0.992778,303.255224,0.063787,0.279137,1.674374,2,True,21
7,RandomForestGini_BAG_L2,-0.139145,-0.150694,log_loss,0.651892,1.000634,303.249269,0.102731,0.286993,1.668419,2,True,20
8,ExtraTreesGini_BAG_L2,-0.141437,-0.143658,log_loss,0.612000,0.959919,302.565630,0.062839,0.246279,0.984780,2,True,23
9,XGBoost_BAG_L2,-0.142561,-0.151890,log_loss,0.643844,0.887140,320.539421,0.094683,0.173499,18.958570,2,True,25
10,LightGBM_BAG_L2,-0.147952,-0.166410,log_loss,0.655804,1.065476,317.141789,0.106643,0.351835,15.560938,2,True,19
11,NeuralNetFastAI_BAG_L1,-0.148331,-0.159531,log_loss,0.327903,0.195441,17.503918,0.327903,0.195441,17.503918,1,True,3
12,CatBoost_BAG_L1,-0.150591,-0.155984,log_loss,0.088703,0.083722,272.328240,0.088703,0.083722,272.328240,1,True,8
13,XGBoost_BAG_L1,-0.155434,-0.170302,log_loss,0.132555,0.434478,11.748693,0.132555,0.434478,11.748693,1,True,11
14,LightGBM_BAG_L1,-0.163646,-0.175614,log_loss,0.210296,0.665820,9.466211,0.210296,0.665820,9.466211,1,True,5
15,LightGBMXT_BAG_L1,-0.169733,-0.179032,log_loss,0.620923,4.006800,15.894285,0.620923,4.006800,15.894285,1,True,4


In [ ]:
# %%

y_pred = predictor.predict_proba(final_test_data.drop(columns=[label]), model=finalModel)

pd.concat([final_test_data[label], y_pred], axis=1).head(10)

,signature,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12
2,0,0.000014,0.000007,0.000006,0.000006,0.000005,0.000879,0.998904,0.000145,0.000006,0.000007,0.000006,0.000007,0.000009
3,4,0.000009,0.000007,0.000009,0.000054,0.000031,0.000010,0.000015,0.000300,0.987022,0.012473,0.000050,0.000007,0.000014
4,2,0.000012,0.000012,0.000011,0.000007,0.000008,0.000025,0.062634,0.935921,0.001331,0.000012,0.000009,0.000008,0.000010
7,2,0.000016,0.000013,0.000009,0.000008,0.000010,0.000016,0.016810,0.983003,0.000077,0.000008,0.000008,0.000009,0.000012
10,-4,0.000012,0.000005,0.000006,0.000262,0.993937,0.005722,0.000011,0.000008,0.000014,0.000007,0.000005,0.000005,0.000006
11,2,0.000018,0.000015,0.000014,0.000015,0.000025,0.000067,0.000212,0.994052,0.005477,0.000056,0.000014,0.000013,0.000023
12,2,0.000010,0.000008,0.000007,0.000007,0.000010,0.000014,0.000131,0.997442,0.002339,0.000007,0.000007,0.000007,0.000010
14,0,0.000010,0.000010,0.000004,0.000003,0.000003,0.000009,0.990966,0.008971,0.000003,0.000004,0.000003,0.000006,0.000007
15,-2,0.000018,0.000012,0.000024,0.000027,0.004975,0.994347,0.000508,0.000022,0.000018,0.000011,0.000011,0.000012,0.000014
18,0,0.000010,0.000004,0.000003,0.000002,0.000002,0.000668,0.999218,0.000070,0.000003,0.000003,0.000002,0.000004,0.000010


In [ ]:
# %%

df_important_features = predictor.feature_importance(final_test_data, model=finalModel)

df_important_features.reset_index()

These features in provided data are not utilized by the predictor and will be ignored: ['Unnamed: 0', 'Symmetry_D3', 'Symmetry_D6', 'Symmetry_D8']
Computing feature importance via permutation shuffling for 14 features using 2500 rows with 5 shuffle sets...
	43.28s	= Expected runtime (8.66s per shuffle set)
	35.38s	= Actual runtime (Completed 5 of 5 shuffle sets)


,index,importance,stddev,p_value,n,p99_high,p99_low
0,meridinal_translation_real,3.531284,0.046082,3.479262e-09,5,3.626168,3.436400
1,longitudinal_translation,1.191761,0.016754,4.685434e-09,5,1.226257,1.157265
2,meridinal_translation_imag,0.937744,0.015089,8.041530e-09,5,0.968812,0.906675
3,short_geodesic_imag_part,0.146822,0.006041,3.430463e-07,5,0.159260,0.134384
4,volume,0.101528,0.005494,1.025203e-06,5,0.112841,0.090215
5,cusp_volume,0.101334,0.002606,5.247624e-08,5,0.106700,0.095967
6,hyperbolic_torsion_degree,0.091396,0.003478,2.510880e-07,5,0.098556,0.084235
7,injectivity_radius,0.053489,0.001391,5.483323e-08,5,0.056353,0.050624
8,short_geodesic_real_part,0.018031,0.001772,1.105273e-05,5,0.021679,0.014382
9,chern_simons,0.012572,0.001135,7.895149e-06,5,0.014910,0.010234


In [ ]:
# %%

final_low_importance_features = df_important_features[df_important_features['importance'] < 0.001].index.to_list()

final_low_importance_features

['hyperbolic_adjoint_torsion_degree']

In [ ]:
# %%

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

import numpy as np
import pandas as pd

from sklearnex import patch_sklearn
patch_sklearn()

label = 'signature'
time_limit = 10*60       # max training time (seconds)
infer_limit = 1/10000    # prediction seconds per row
finalModel = 'CatBoost_BAG_L2'

data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'

train_data = TabularDataset(f'{data_url}train.csv')
test_data = TabularDataset(f'{data_url}test.csv')

pruning_test_data = test_data.sample(frac=0.5, random_state=2024)
final_test_data = test_data.drop(pruning_test_data.index)

train_data.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/train.csv | Columns = 19 / 19 | Rows = 10000 -> 10000
Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/test.csv | Columns = 19 / 19 | Rows = 5000 -> 5000


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# %% 

# Detect features to prune
np.random.seed(2024)

pruning_predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)
pruning_predictor.fit(train_data, presets='medium_quality', time_limit = time_limit)  

pruning_leaders = pruning_predictor.leaderboard(pruning_test_data)
pruning_leaders

No path specified. Models will be saved in: "AutogluonModels\ag-20240826_090242"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       17.14 GB / 31.75 GB (54.0%)
Disk Space Avail:   756.72 GB / 952.44 GB (79.5%)
Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 600s
AutoGluon will save models to "AutogluonModels\ag-20240826_090242"
Train Data Rows:    10000
Train Data Columns: 18
Label Column:       signature
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 13
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    17552.71 MB
	Train Data (Original)  Memory Usage: 1.37 MB (0.0% of available memory)
	Inferring data type of each feature based

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-0.152299,-0.129025,log_loss,0.282399,0.049865,104.015484,0.003986,0.001003,0.299377,2,True,14
1,CatBoost,-0.159601,-0.150310,log_loss,0.036481,0.007678,79.909065,0.036481,0.007678,79.909065,1,True,8
2,NeuralNetFastAI,-0.175997,-0.166266,log_loss,0.051285,0.018448,11.387197,0.051285,0.018448,11.387197,1,True,3
3,XGBoost,-0.182194,-0.144754,log_loss,0.159817,0.009967,4.327720,0.159817,0.009967,4.327720,1,True,11
4,LightGBM,-0.190363,-0.161321,log_loss,0.048863,0.009972,2.506484,0.048863,0.009972,2.506484,1,True,5
5,RandomForestEntr,-0.199600,-0.194874,log_loss,0.153096,0.047135,1.050660,0.153096,0.047135,1.050660,1,True,7
6,NeuralNetTorch,-0.199837,-0.163288,log_loss,0.018448,0.010474,85.494705,0.018448,0.010474,85.494705,1,True,12
7,LightGBMXT,-0.202060,-0.178918,log_loss,0.128386,0.030413,4.047957,0.128386,0.030413,4.047957,1,True,4
8,RandomForestGini,-0.211661,-0.206937,log_loss,0.181019,0.046126,0.981958,0.181019,0.046126,0.981958,1,True,6
9,ExtraTreesEntr,-0.263108,-0.267746,log_loss,0.188714,0.063318,0.806900,0.188714,0.063318,0.806900,1,True,10


In [ ]:
# %%

df_pruning_features = pruning_predictor.feature_importance(pruning_test_data)
low_importance_features = df_pruning_features.query("importance < 0.001").index.to_list()
low_importance_features

These features in provided data are not utilized by the predictor and will be ignored: ['Symmetry_D8']
Computing feature importance via permutation shuffling for 17 features using 2500 rows with 5 shuffle sets...
	29.42s	= Expected runtime (5.88s per shuffle set)
c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-packages\sklearn\metrics\_classification.py:2969: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-packages\sklearn\metrics\_classification.py:2969: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-packages\sklearn\metrics\_classification.py:2969: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
c:\Users\chiuw\miniconda3\envs\env_AutoGluon_202408\lib\site-

['Symmetry_D6', 'Unnamed: 0', 'Symmetry_D3']

In [ ]:
# %%

final_train_data = train_data.drop(columns=low_importance_features)

In [ ]:
# %%

custom_hyperparameters = get_hyperparameter_config('zeroshot') # ['default', 'zeroshot', 'light', 'very_light', 'toy', 'multimodal']

custom_hyperparameters['LR'] = [
    {'multi_class':'multinomial', 'penalty':None, 'tol':1e-6, 'max_iter':10000,
        'ag_args': {'name_suffix': 'Base'}},
    {'multi_class':'multinomial', 'penalty':'l2', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Ridge'}},
    {'multi_class':'multinomial', 'penalty':'l1', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'Lasso'}},
    {'multi_class':'multinomial', 'penalty':'elasticnet', 'tol':1e-6, 'max_iter':10000, 'C':0.1,
        'ag_args': {'name_suffix': 'ElasticNet'}}
    ]

custom_preset = {'auto_stack': True,
                'dynamic_stacking': False,
                'hyperparameters':custom_hyperparameters,
                'refit_full': False,
                'set_best_to_refit_full': False,
                'save_bag_folds': True,
                'time_limit': time_limit,
                'infer_limit': infer_limit}

In [ ]:
# %%

np.random.seed(2024)

predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)

predictor.fit(final_train_data, presets=custom_preset)  

No path specified. Models will be saved in: "AutogluonModels\ag-20240826_090612"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       17.30 GB / 31.75 GB (54.5%)
Disk Space Avail:   756.18 GB / 952.44 GB (79.4%)
Presets specified: [{'auto_stack': True, 'dynamic_stacking': False, 'hyperparameters': {'NN_TORCH': [{}, {'activation': 'elu', 'dropout_prob': 0.10077639529843717, 'hidden_size': 108, 'learning_rate': 0.002735937344002146, 'num_layers': 4, 'use_batchnorm': True, 'weight_decay': 1.356433327634438e-12, 'ag_args': {'name_suffix': '_r79', 'priority': -2}}, {'activation': 'elu', 'dropout_prob': 0.11897478034205347, 'hidden_size': 213, 'learning_rate': 0.0010474382260641949, 'num_layers': 4, 'use_batchnorm': False, 'weight_decay': 5.594471067786272e-10, 'ag_args': {'name_suf

In [ ]:
# %%

predictor.persist(models='all', max_memory=0.5) # improves prediction time, consumes more memory

df_leaders = predictor.leaderboard(final_test_data)

df_leaders.head(40)

Persisting 26 models in memory. Models will require 7.64% of memory.


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.129586,-0.130727,log_loss,0.777545,1.518218,419.589747,0.003987,0.003987,0.339903,3,True,26
1,LightGBMXT_BAG_L2,-0.130877,-0.174001,log_loss,0.738529,1.345126,318.654713,0.184384,0.616553,10.565393,2,True,18
2,ExtraTreesEntr_BAG_L2,-0.131844,-0.141450,log_loss,0.617008,0.974802,308.867182,0.062863,0.246229,0.777862,2,True,24
3,CatBoost_BAG_L2,-0.134833,-0.143815,log_loss,0.582043,0.772432,415.800249,0.027898,0.043859,107.710929,2,True,22
4,WeightedEnsemble_L2,-0.135982,-0.139773,log_loss,0.555143,0.733556,308.448369,0.000998,0.004982,0.359048,2,True,16
5,NeuralNetFastAI_BAG_L2,-0.137136,-0.150249,log_loss,0.891934,0.912464,330.335978,0.337789,0.183891,22.246658,2,True,17
6,RandomForestEntr_BAG_L2,-0.137723,-0.140167,log_loss,0.616943,0.965480,309.701736,0.062798,0.236907,1.612416,2,True,21
7,RandomForestGini_BAG_L2,-0.139145,-0.150694,log_loss,0.666847,0.975816,309.689365,0.112702,0.247243,1.600045,2,True,20
8,ExtraTreesGini_BAG_L2,-0.141437,-0.143658,log_loss,0.619999,0.987236,309.148637,0.065854,0.258663,1.059317,2,True,23
9,XGBoost_BAG_L2,-0.142626,-0.151890,log_loss,0.636868,0.900995,326.620855,0.082723,0.172422,18.531535,2,True,25


In [ ]:
# %%

df_leaders.tail(20)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
6,RandomForestEntr_BAG_L2,-0.137723,-0.140167,log_loss,0.616943,0.965480,309.701736,0.062798,0.236907,1.612416,2,True,21
7,RandomForestGini_BAG_L2,-0.139145,-0.150694,log_loss,0.666847,0.975816,309.689365,0.112702,0.247243,1.600045,2,True,20
8,ExtraTreesGini_BAG_L2,-0.141437,-0.143658,log_loss,0.619999,0.987236,309.148637,0.065854,0.258663,1.059317,2,True,23
9,XGBoost_BAG_L2,-0.142626,-0.151890,log_loss,0.636868,0.900995,326.620855,0.082723,0.172422,18.531535,2,True,25
10,LightGBM_BAG_L2,-0.148066,-0.166410,log_loss,0.660788,1.073423,322.721689,0.106643,0.344850,14.632369,2,True,19
11,NeuralNetFastAI_BAG_L1,-0.148331,-0.159531,log_loss,0.328900,0.195025,19.272645,0.328900,0.195025,19.272645,1,True,3
12,CatBoost_BAG_L1,-0.150591,-0.155984,log_loss,0.091693,0.099856,277.449635,0.091693,0.099856,277.449635,1,True,8
13,XGBoost_BAG_L1,-0.155434,-0.170302,log_loss,0.133552,0.433692,11.367040,0.133552,0.433692,11.367040,1,True,11
14,LightGBM_BAG_L1,-0.163646,-0.175614,log_loss,0.203320,0.646026,9.554695,0.203320,0.646026,9.554695,1,True,5
15,LightGBMXT_BAG_L1,-0.169733,-0.179032,log_loss,0.613827,3.163609,17.049018,0.613827,3.163609,17.049018,1,True,4


In [ ]:
# %%

y_pred = predictor.predict_proba(final_test_data.drop(columns=[label]), model=finalModel)

pd.concat([final_test_data[label], y_pred], axis=1).head(10)

,signature,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12
2,0,0.000012,0.000013,0.000015,0.000017,0.000056,0.001736,0.997259,0.000795,0.000044,0.000016,0.000013,0.000013,0.000012
3,4,0.000043,0.000045,0.000058,0.000085,0.000102,0.000133,0.000198,0.001997,0.986699,0.010357,0.000173,0.000059,0.000051
4,2,0.000030,0.000035,0.000040,0.000047,0.000098,0.000388,0.012358,0.985263,0.001563,0.000069,0.000045,0.000033,0.000031
7,2,0.000016,0.000017,0.000020,0.000021,0.000051,0.000191,0.005572,0.993209,0.000819,0.000030,0.000021,0.000018,0.000016
10,-4,0.000023,0.000028,0.000060,0.000825,0.995206,0.003468,0.000169,0.000075,0.000048,0.000028,0.000025,0.000023,0.000021
11,2,0.000021,0.000023,0.000024,0.000029,0.000056,0.000155,0.001869,0.992630,0.005030,0.000083,0.000032,0.000024,0.000024
12,2,0.000017,0.000019,0.000021,0.000021,0.000042,0.000105,0.001296,0.994627,0.003730,0.000056,0.000028,0.000020,0.000019
14,0,0.000014,0.000015,0.000018,0.000019,0.000042,0.000560,0.996379,0.002819,0.000070,0.000019,0.000017,0.000015,0.000014
15,-2,0.000016,0.000018,0.000022,0.000047,0.002247,0.995322,0.002142,0.000090,0.000035,0.000019,0.000014,0.000015,0.000014
18,0,0.000015,0.000018,0.000021,0.000022,0.000074,0.001691,0.996975,0.001053,0.000056,0.000021,0.000019,0.000018,0.000016


In [ ]:
# %%

df_important_features = predictor.feature_importance(final_test_data, model=finalModel)

df_important_features.reset_index()

These features in provided data are not utilized by the predictor and will be ignored: ['Unnamed: 0', 'Symmetry_D3', 'Symmetry_D6', 'Symmetry_D8']
Computing feature importance via permutation shuffling for 14 features using 2500 rows with 5 shuffle sets...
	45.95s	= Expected runtime (9.19s per shuffle set)
	34.47s	= Actual runtime (Completed 5 of 5 shuffle sets)


,index,importance,stddev,p_value,n,p99_high,p99_low
0,meridinal_translation_real,3.114202,0.034937,1.900415e-09,5,3.186138,3.042267
1,longitudinal_translation,1.236546,0.017188,4.478335e-09,5,1.271936,1.201156
2,meridinal_translation_imag,1.044602,0.016647,7.737172e-09,5,1.078878,1.010325
3,short_geodesic_imag_part,0.195665,0.008501,4.265496e-07,5,0.213170,0.178161
4,volume,0.134064,0.009151,2.588984e-06,5,0.152906,0.115222
5,cusp_volume,0.131414,0.004177,1.223273e-07,5,0.140015,0.122814
6,hyperbolic_torsion_degree,0.109616,0.004120,2.391116e-07,5,0.118100,0.101133
7,injectivity_radius,0.061010,0.002605,3.980877e-07,5,0.066374,0.055646
8,short_geodesic_real_part,0.015922,0.001936,2.574241e-05,5,0.019909,0.011935
9,chern_simons,0.009109,0.001586,1.060211e-04,5,0.012375,0.005843


In [ ]:
# %%

final_low_importance_features = df_important_features[df_important_features['importance'] < 0.001].index.to_list()

final_low_importance_features

['hyperbolic_adjoint_torsion_degree']

2024-08-26 02:20:00,117	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-26 02:20:00,122	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-26 02:20:00,124	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-26 02:20:04,609	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.
2024-08-26 02:20:07,368	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UN